In [1]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from random import randint

In [14]:
def train_test_split(split_part):
    og_data = np.load('og_data.npy')
    labels = np.load('labels.npy')
    
    # Label encoder
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels = le.transform(labels)
    x_train, x_test, y_train, y_test = [], [], [], []

    smin = -1
    smax = 1
    x = og_data
    xmax, xmin = x.max(), x.min()
    norm_data = (x - xmin)*(smax - smin)/(xmax - xmin) + smin

    # Sorting the data
    sorted_index = np.argsort(labels)
    sorted_labels = labels[sorted_index]
    sorted_og_data = og_data[sorted_index]
    sorted_norm_data = norm_data[sorted_index]

    # split the data into train and test
    test_random_indices = []
    for i in range(sorted_labels.shape[0]/split_part):
        index = i*split_part + randint(0, split_part - 1)
        test_random_indices.append(index)

    x_train = np.delete(sorted_norm_data, test_random_indices, 0)
    x_test = sorted_norm_data[test_random_indices]
    y_train = np.delete(sorted_labels, test_random_indices)
    y_test = sorted_labels[test_random_indices]
    
    return x_train, x_test, y_train, y_test

In [10]:
x_train, x_test, y_train, y_test = train_test_split(6)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

('x_train shape:', (300, 21764))
(300, 'train samples')
(60, 'test samples')


In [25]:
results = []
results_top = []
for i in range(1):
    print '{0} iteration started --> '.format(i),
    clf = LinearSVC()
    x_train, x_test, y_train, y_test = train_test_split(6)
    clf.fit(x_train, y_train)
    results.append(clf.score(x_test, y_test))
    
    top_r = []
    for j in [1, 3, 5, 10]:
        predicted_correct = 0
        for k in range(60):
            predicted_lbl = clf.predict(x_test[k].reshape(1, -1))
            decision_lbl = clf.decision_function(x_test[k].reshape(1, -1))[0]

            top_class = j
            top_n = np.argsort(decision_lbl)[::-1][:top_class]

            if y_test[k] in top_n:
                predicted_correct += 1.0/60
        top_r.append(predicted_correct)
    results_top.append(top_r)
    print 'completed'

0 iteration started -->  completed


In [28]:
results_t = np.array(results_top)

results_t = np.average(results_t, axis=0)

Matrics = ['Top_1', 'Top_3', 'Top_5', 'Top_10']

LinearSVC_results = zip(Matrics, results_t)

print "With normalisation to data"
for i in LinearSVC_results:
    print 'LinearSVC result for %s is %s.' % (i)

With normalisation to data
LinearSVC result for Top_1 is 0.16666666666666666.
LinearSVC result for Top_3 is 0.36666666666666664.
LinearSVC result for Top_5 is 0.5833333333333335.
LinearSVC result for Top_10 is 0.7166666666666672.


In [265]:
predicted_correct = 0
for i in range(60):
    predicted_lbl = clf.predict(x_test[i].reshape(1, -1))
    decision_lbl = clf.decision_function(x_test[i].reshape(1, -1))[0]

    top_class = 3
    top_n = np.argsort(decision_lbl)[::-1][:top_class]
    
    if y_test[i] in top_n:
        predicted_correct += 1.0/60

[0.25,
 0.2833333333333333,
 0.26666666666666666,
 0.31666666666666665,
 0.2,
 0.2,
 0.2833333333333333,
 0.15,
 0.21666666666666667,
 0.16666666666666666,
 0.21666666666666667,
 0.2,
 0.21666666666666667,
 0.3,
 0.3,
 0.13333333333333333,
 0.26666666666666666,
 0.18333333333333332,
 0.2,
 0.23333333333333334,
 0.21666666666666667,
 0.21666666666666667,
 0.26666666666666666,
 0.21666666666666667,
 0.2,
 0.2833333333333333,
 0.2,
 0.15,
 0.15,
 0.23333333333333334,
 0.26666666666666666,
 0.2833333333333333,
 0.21666666666666667,
 0.25,
 0.3333333333333333,
 0.23333333333333334,
 0.15,
 0.21666666666666667,
 0.2833333333333333,
 0.31666666666666665,
 0.23333333333333334,
 0.21666666666666667,
 0.25,
 0.23333333333333334,
 0.2,
 0.31666666666666665,
 0.21666666666666667,
 0.31666666666666665,
 0.21666666666666667,
 0.21666666666666667]

In [37]:
count = 0
lbs = clf.classes_
for i in range(60):
    pb = clf.predict(x_test[0].reshape(1, -1))
    probs = clf.predict_proba(x_test[i].reshape(1, -1))
    best_n = np.argsort(probs, axis=1)[0][::-1]
    best_n = best_n[:30]
    predicted_lbs = list()
    lbl_probs = list()
    for j in range(1):
        predicted_lbs.append(lbs[best_n[j]])
        lbl_probs.append(probs[0][best_n[j]])
    predictions = zip(predicted_lbs, lbl_probs)
    
    if y_test[i] in predicted_lbs:
        print predictions, y_test[i], pb
        count += 1

[('train', 0.026734342151736153)] train ['coat']
